# Training Model Template

## Details about implementation

This Model is implemented from a YouTube Tutorial, Its simple and efficient. This notebook includes all the necessary code and visualization that may help understand the model better.

In [ ]:
MODEL_NAME = "plant_village_cnn_model1"

### Importing Libraries

In [1]:
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import image_dataset_from_directory, load_img, img_to_array

In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay
)

### Importing Dataset

#### Dataset Preparation

In [ ]:
# Paths to the dataset folders
TRAIN_DIR = "../datasets/cropped_plant_village_dataset/train"
VALID_DIR = "../datasets/cropped_plant_village_dataset/valid"
SAMPLE_IMAGE = "../datasets/cropped_plant_village_dataset/sample_image.JPG"

##### Training Set

In [ ]:
training_set = image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

##### Validation Set

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

#### Dataset Details

In [ ]:
# Function to count images in each class
def count_images_in_classes(dataset_dir):
    """
    Counts the number of images in each class within a dataset directory.

    Args:
        dataset_dir (str): The path to the dataset directory.

    Returns:
        dict: A dictionary where the keys are the class names and the values are the number of images in each class.
    """
    class_counts = {}
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts


# Count images in training and validation sets
train_class_counts = count_images_in_classes(TRAIN_DIR)
valid_class_counts = count_images_in_classes(VALID_DIR)

# Create a DataFrame for better visualization
df = pd.DataFrame(
    {
        "Class": list(train_class_counts.keys()),
        "Training Images": list(train_class_counts.values()),
        "Validation Images": [
            valid_class_counts.get(cls, 0) for cls in train_class_counts.keys()
        ],
    }
)

In [ ]:
# Display the table
print(df)

In [ ]:
# Plot the class distribution
df.plot(
    x="Class", kind="bar", stacked=True, figsize=(15, 6), title="Class Distribution"
)
plt.ylabel("Number of Images")
plt.xlabel("Class")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Function to visualize one image per class
def visualize_sample_images(dataset_dir):
    """
    Visualizes one sample image per class in the dataset directory.

    Args:
        dataset_dir (str): The path to the dataset directory.

    Displays a grid of images, with one image per class, using matplotlib.
    """
    class_names = os.listdir(dataset_dir)
    class_names.sort()  # Sort for consistent order

    plt.figure(figsize=(15, 10))
    for i, class_name in enumerate(class_names, start=1):
        class_path = os.path.join(dataset_dir, class_name)
        image_path = os.path.join(
            class_path, os.listdir(class_path)[0]
        )  # Get the first image in the class
        img = plt.imread(image_path)

        plt.subplot(4, 5, i)  # Adjust grid size for the number of classes
        plt.imshow(img)
        plt.title(class_name)
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize sample images from training set
visualize_sample_images(TRAIN_DIR)

### Image Preprocessing

#### Normalization

In [ ]:
normalize = tf.keras.layers.Rescaling(1.0 / 255)

# Apply normalization on both Training and Validation set
training_set = training_set.map(lambda x, y: (normalize(x), y))
validation_set = validation_set.map(lambda x, y: (normalize(x), y))

#### Image Augmentation

In [ ]:
data_augmentation = Sequential(
    [
        # Brightness adjustment
        tf.keras.layers.RandomBrightness(factor=0.2),  # Adjust brightness by ±20%

        # Contrast adjustment
        tf.keras.layers.RandomContrast(factor=0.2),  # Adjust contrast by ±20%

        # Rotation
        tf.keras.layers.RandomRotation(factor=0.1),  # Rotate by ±10% (36°)
        
        # Horizontal and vertical flips
        tf.keras.layers.RandomFlip(
            mode="horizontal_and_vertical"
        ),  # Flip both horizontally and vertically
        
        # Zoom
        tf.keras.layers.RandomZoom(
            height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)
        ),  # Zoom in/out by 20%

        # Gaussian noise
        tf.keras.layers.Lambda(
            lambda x: x + tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=0.01)
        ),  # Add Gaussian noise

    ]
)

# Apply augmentation to the training set
augmented_training_set = training_set.map(lambda x, y: (data_augmentation(x), y))

#### Image Enhancement (not implemented)

In [ ]:
# Image Enhancements have not strongly proven to increase the accuracy

#### Image Preprocessing Details

##### Augmentation Details

In [ ]:
# Test the augmentation pipeline with individual augmentations
def visualize_individual_augmentations(image_path):
    """
    Visualizes the effect of individual augmentations on an input image.

    Args:
        image_path (str): The path to the input image.

    Applies a series of individual augmentations to the input image and displays the results in a grid.
    """
    # Load and preprocess the image
    image = load_img(image_path, target_size=(128, 128))  # Adjust to your image size
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension

    # Define individual augmentation layers
    augmentations = [
        ("Original", None),
        ("Random Brightness", tf.keras.layers.RandomBrightness(factor=0.2)),
        ("Random Contrast", tf.keras.layers.RandomContrast(factor=0.2)),
        ("Random Rotation", tf.keras.layers.RandomRotation(factor=0.1)),
        ("Random Flip", tf.keras.layers.RandomFlip(mode="horizontal_and_vertical")),
        (
            "Random Zoom",
            tf.keras.layers.RandomZoom(
                height_factor=(-0.2, 0.2), width_factor=(-0.2, 0.2)
            ),
        ),
        (
            "Gaussian Noise",
            tf.keras.layers.Lambda(
                lambda x: x + tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=0.01)
            ),
        ),
    ]

    # Apply each augmentation and plot
    plt.figure(figsize=(20, 5))
    for i, (title, layer) in enumerate(augmentations, start=1):
        if layer is None:
            augmented_image = image_array[0]
        else:
            augmented_image = layer(image_array)[0]

        plt.subplot(1, len(augmentations), i)
        plt.imshow(augmented_image.numpy())
        plt.title(title)
        plt.axis("off")

    plt.show()

In [ ]:
# Visualize individual augmentation techniques
visualize_individual_augmentations(SAMPLE_IMAGE)

In [ ]:
# Test the augmentation pipeline with a sample image
def visualize_augmentation(image_path):
    """
    Visualizes the effect of the augmentation pipeline on a sample image.

    Args:
        image_path (str): The path to the input image.

    Applies the augmentation pipeline to the input image and displays the original image alongside 5 augmented versions.
    """

    # Load and preprocess the image
    image = load_img(image_path, target_size=(128, 128))  # Adjust to your image size
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = tf.expand_dims(image_array, axis=0)  # Add batch dimension

    # Apply augmentations
    augmented_images = [data_augmentation(image_array)[0] for _ in range(5)]

    # Plot original and augmented images
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 6, 1)
    plt.imshow(image_array[0])
    plt.title("Original")
    plt.axis("off")

    for i, aug_img in enumerate(augmented_images, start=2):
        plt.subplot(1, 6, i)
        plt.imshow(aug_img.numpy())
        plt.title(f"Augmented {i-1}")
        plt.axis("off")
    plt.show()

In [ ]:
# Visualize the Original vs Augmented Image
visualize_augmentation(SAMPLE_IMAGE)

### Training Model

#### Building Model

In [ ]:
def add_conv_block(
    model,
    filters,
    kernel_size=3,
    pool_size=2,
    strides=2,
    activation="relu",
    padding="same",
):
    """
    Adds a convolutional block to the model, consisting of:
    - Conv2D layer
    - Conv2D layer
    - MaxPooling layer

    Parameters:
        model: Sequential model to which the block is added.
        filters: Number of filters for Conv2D layers.
        kernel_size: Size of the convolutional kernel (default: 3).
        pool_size: Pool size for MaxPooling (default: 2).
        strides: Strides for MaxPooling (default: 2).
        activation: Activation function for Conv2D layers (default: 'relu').
        padding: Padding for Conv2D layers (default: 'same').
    """
    model.add(
        Conv2D(
            filters=filters,
            kernel_size=kernel_size,
            padding=padding,
            activation=activation,
        )
    )
    model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation))
    model.add(MaxPool2D(pool_size=pool_size, strides=strides))

In [ ]:
# Initialize the model
model = Sequential()

In [ ]:
# Input layer and first convolutional block
model.add(
    Conv2D(
        filters=32,
        kernel_size=3,
        padding="same",
        activation="relu",
        input_shape=[128, 128, 3],
    )
)
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
# Add subsequent convolutional blocks using the function
add_conv_block(model, filters=64)
add_conv_block(model, filters=128)
add_conv_block(model, filters=256)
add_conv_block(model, filters=512)
add_conv_block(model, filters=1024)

In [ ]:
# Add the fully connected layers
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(units=1500, activation="relu"))
model.add(Dropout(0.4))

In [ ]:
# Output Layer
model.add(Dense(units=38, activation="softmax"))

#### Compiling Model

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

#### Training Model

In [ ]:
training_history = model.fit(x=training_set, validation_data=validation_set, epochs=10)

#### Evaluating Model

In [ ]:
# Training set Accuracy
train_loss, train_acc = model.evaluate(training_set)
print("Training accuracy:", train_acc)

In [ ]:
# Validation set Accuracy
val_loss, val_acc = model.evaluate(validation_set)
print("Validation accuracy:", val_acc)

#### Saving Model

In [ ]:
model.save(f'../models/{MODEL_NAME}.keras')

In [ ]:
# Save the training history
with open("training_history.json", "w") as f:
    json.dump(training_history.history, f)

#### Model Details

##### Model Architecture

In [ ]:
model.summary()

##### Model Performance Metrics

In [ ]:
# Map class indices to class names
class_names = validation_set.class_names
print(f"Class Names: {class_names}")

In [ ]:
# Get true labels
y_true = np.concatenate([y.numpy() for _, y in validation_set], axis=0)
y_true = np.argmax(y_true, axis=1)  # Convert one-hot encoding to class indices

In [ ]:
# Predict labels using the trained model
y_pred_probs = model.predict(validation_set)  # Replace 'model' with your trained model
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
# Generate the classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:")
print(report)

##### Model Confusion Matrix

In [ ]:
def plot_confusion_matrix(model, validation_set, class_names):
    """
    Plots the confusion matrix for a given model and validation dataset.

    Parameters:
        model: Trained model.
        validation_set: Validation dataset (normalized and preprocessed).
        class_names: List of class labels.
    """
    # Get true labels and predictions
    true_labels = np.concatenate([y for x, y in validation_set], axis=0)
    predicted_probs = model.predict(validation_set)
    predicted_labels = np.argmax(predicted_probs, axis=1)
    true_labels = np.argmax(true_labels, axis=1)

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)

    # Plot confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    plt.figure(figsize=(10, 8))
    disp.plot(cmap=plt.cm.Blues, values_format="d")
    plt.title("Confusion Matrix")
    plt.show()

In [ ]:
# Example usage
class_names = validation_set.class_names  # Assuming your dataset has class names
plot_confusion_matrix(model, validation_set, class_names)

##### Train | Valid Accuracy & Loss graph

In [ ]:
def plot_training_history(history):
    """
    Plots training and validation accuracy and loss graphs.

    Parameters:
        history: The History object returned by model.fit().
    """
    # Extract metrics
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1, len(acc) + 1)

    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label="Training Accuracy")
    plt.plot(epochs, val_acc, label="Validation Accuracy")
    plt.title("Training and Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label="Training Loss")
    plt.plot(epochs, val_loss, label="Validation Loss")
    plt.title("Training and Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the graphs
plot_training_history(training_history)

## Conclusion

This notebook provides a comprehensive template for training a CNN model for image classification. It includes dataset preparation, preprocessing, augmentation, model building, training, evaluation, and visualization. Once executed, it is expected to deliver a functional model for plant disease classification. However, adding hyperparameter tuning and overfitting mitigation strategies could further improve its utility.

Areas for Improvement:

The notebook lacks details about hyperparameter tuning. Including experiments with different learning rates or optimizers could enhance it.
While the augmentation techniques are detailed, their effectiveness on model performance is not quantified.
Consider adding a section on overfitting prevention strategies like early stopping or learning rate scheduling.